In [1]:
from scipy.io import loadmat

import h5py
import xarray as xr
import numpy as np

#PLOTTING
import cartopy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
import matplotlib.ticker as mticker

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#resampling to grid
from pyresample.geometry import AreaDefinition
from pyresample.geometry import GridDefinition
from pyresample import image, geometry, load_area, save_quicklook, SwathDefinition, area_def2basemap
from pyresample.kd_tree import resample_nearest
from pyresample.utils import check_and_wrap
from scipy import spatial

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

import glob

#where to find the data
adir_data= './../../data/orbit/'
adir_figs= './../../data/orbit/figures/'

# add land mask
#get bathymetry from ETOPO1
fname_topo = './../../data/topo/ETOPO1_Ice_g_gmt4.grd'
ds = xr.open_dataset(fname_topo)
ds_topo = ds.rename_dims({'x':'lon','y':'lat'}).rename({'x':'lon','y':'lat'})
tem = ds_topo.attrs
ds_topo = ds_topo.rename({'z':'etopo_depth'})
ds_topo.etopo_depth.attrs=tem
_, index = np.unique(ds_topo['lon'], return_index=True)
ds_topo = ds_topo.isel(lon=index)
_, index = np.unique(ds_topo['lat'], return_index=True)
ds_topo = ds_topo.isel(lat=index)

In [3]:
#import sys
#sys.path.append('C:/Users/gentemann/Desktop/git_python/ECCOv4-py')
#import ecco_v4_py as ecco
ds_ecco = xr.open_dataset('./../data/ecco2011_11_19_latlonTheta.nc')
ds_ecco

<xarray.Dataset>
Dimensions:  (lat: 4320, lon: 8640)
Coordinates:
  * lat      (lat) float64 -89.98 -89.94 -89.9 -89.85 ... 89.85 89.9 89.94 89.98
  * lon      (lon) float64 0.02083 0.0625 0.1042 0.1458 ... 359.9 359.9 360.0
Data variables:
    Theta    (lat, lon) float32 ...

In [1]:
ds_topo2 = ds_topo.interp(lat=np.arange(-90,90,0.1),lon=np.arange(-180,180,0.1))
ds_topo2.etopo_depth.plot()

NameError: name 'ds_topo' is not defined

In [ ]:
#get all filenames
filenames = glob.glob(adir_data+'*.nc')
ds = xr.open_mfdataset(filenames)
da2 = ds.sec
da2

In [ ]:
da2a = da2.fillna(0)
dy = da2a[:,:,0:16].sum(dim=['orbit'])
dy.plot(vmin=0,vmax=5)

In [ ]:
da2a = da2.fillna(0)
dy = da2a[:,:,0:16].sum(dim=['orbit'])
dy = dy.where(ds_topo2.etopo_depth<0)
dy.plot(vmin=0,vmax=5)

In [ ]:
ds2 = da2#.isel(lat=0)
#dy = da2a[:,:,0]+da2a[:,:,1]+da2a[:,:,16]+da2a[:,:,32]
#dy.plot(c=
#plt.plot(da2[900,:,0],linewidth=15)
#plt.plot(da2[900,:,1],linewidth=15)
#plt.plot(da2[900,:,2],linewidth=15)
#plt.plot(da2[900,:,16],linewidth=10)
#plt.plot(da2[900,:,17],linewidth=10)
#plt.plot(da2[900,:,18],linewidth=10)
#plt.plot(da2[900,:,32],linewidth=5)
#plt.plot(da2[900,:,33],linewidth=5)
#plt.plot(da2[900,:,34],linewidth=5)

plt.plot(da2[900,:,0],'r*')
plt.plot(da2[900,:,1],'r*')
plt.plot(da2[900,:,2],'r*')
plt.plot(da2[900,:,16]+.0001,'b*')
plt.plot(da2[900,:,17]+.0001,'b*')
plt.plot(da2[900,:,18]+.0001,'b*')
plt.plot(da2[900,:,32]+.0002,'g*')
plt.plot(da2[900,:,33]+.0002,'g*')
plt.plot(da2[900,:,34]+.0002,'g*')


plt.ylim(.995,1.005)
plt.xlim(1550,2300)
plt.savefig(adir_figs+'data_cov.png', dpi=100)

In [ ]:
#can put in any mask from here: https://neo.sci.gsfc.nasa.gov/view.php?datasetId=MOD_NDVI_M&date=2020-11-01
import cartopy.feature as cfeature
land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor=cfeature.COLORS['land'])
img = plt.imread('./../../data/background_images/bluemarble_10km_august.png')
img_extent = (-180, 180, -90, 90)
#plt.imshow(img, origin='upper', extent=img_extent)
img2 = np.flip(img,0)
for i in range(3):
    img2[:,:,i] = np.where(ds_topo2.etopo_depth.data>0,img2[:,:,i],np.nan)
img2 = np.flip(img2,0)
img_white = np.where(np.isfinite(img2),img2,1.0)
plt.imshow(img3, origin='upper', extent=img_extent)

In [ ]:
fig = plt.figure(figsize=(10, 8))
coord1 = 121
coord2 = 122
da2a = da2.fillna(0)
dy = da2a[:,:,0:32].sum(dim=['orbit'])
dy = dy.where(ds_topo2.etopo_depth<0)  #mask land regions
dy = dy.where(dy>0,np.nan)
ax = plt.subplot(coord1,projection=ccrs.Orthographic(-20, 10))
ax.imshow(img2, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
dy.plot(ax=ax, transform=ccrs.PlateCarree(),cmap='jet',vmin=0,vmax=5,add_colorbar=False)

dy = da2a[:,:,0:48].sum(dim=['orbit'])
dy = dy.where(ds_topo2.etopo_depth<0)  #mask land regions
dy = dy.where(dy>0,np.nan)
ax2 = plt.subplot(coord2,projection=ccrs.Orthographic(-20, 10))
ax2.imshow(img3, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
im=dy.plot(ax=ax2, transform=ccrs.PlateCarree(),cmap='jet',vmin=0,vmax=5,add_colorbar=False)
#ax2.coastlines()
#ax2.stock_img()
#ax2.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
#ax2.add_feature(land_50m, zorder=100, edgecolor='k')
plt.savefig(adir_figs+'2dy_3dy_numA.png', dpi=100)

In [ ]:
fig = plt.figure(figsize=(10, 8))
coord1 = 121
coord2 = 122
da2a = da2.fillna(0)
dy = da2a[:,:,0:32].sum(dim=['orbit'])
dy = dy.where(ds_topo2.etopo_depth<0)  #mask land regions
dy = dy.where(dy>0,np.nan)
ax = plt.subplot(coord1,projection=ccrs.Orthographic(180, 10))
ax.imshow(img2, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
dy.plot(ax=ax, transform=ccrs.PlateCarree(),cmap='jet',vmin=0,vmax=5,add_colorbar=False)

dy = da2a[:,:,0:48].sum(dim=['orbit'])
dy = dy.where(ds_topo2.etopo_depth<0)  #mask land regions
dy = dy.where(dy>0,np.nan)
ax2 = plt.subplot(coord2,projection=ccrs.Orthographic(180, 10))
ax2.imshow(img2, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
im=dy.plot(ax=ax2, transform=ccrs.PlateCarree(),cmap='jet',vmin=0,vmax=5,add_colorbar=False)
#ax2.coastlines()
#ax2.stock_img()
#ax2.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
#ax2.add_feature(land_50m, zorder=100, edgecolor='k')
plt.savefig(adir_figs+'2dy_3dy_num.png', dpi=100)

In [ ]:
da2a = da2.fillna(0)
dy1 = da2a[:,:,0:16].max(dim=['orbit']).sum(dim=['lon'])/3600
dy2 = da2a[:,:,0:32].max(dim=['orbit']).sum(dim=['lon'])/3600
dy3 = da2a[:,:,0:48].max(dim=['orbit']).sum(dim=['lon'])/3600
plt.barh(dy3.lat,dy3.data*100,color='green')
plt.barh(dy2.lat,dy2.data*100,color='blue')
plt.barh(dy1.lat,dy1.data*100,color='cyan')
plt.xlim(0,100)
plt.ylim(-85,85)


In [ ]:
# calc %
R = 6.37e6
# we know already that the spacing of the points is one degree latitude
dϕ = np.deg2rad(1.)
dλ = np.deg2rad(1.)
dA = R**2 * dϕ * dλ * np.cos(np.deg2rad(dy.lat))
#dA.where(dy.notnull())
dy = da2a[:,:,0:16].max(dim=['orbit'])
pixel_area = dA.where(dy.notnull())
total_ocean_area = pixel_area.sum(dim=('lon', 'lat'))
sst_weighted_mean = (dy * pixel_area).sum(dim=('lon', 'lat')) / total_ocean_area
print(sst_weighted_mean)
dy = da2a[:,:,0:32].max(dim=['orbit'])
pixel_area = dA.where(dy.notnull())
total_ocean_area = pixel_area.sum(dim=('lon', 'lat'))
sst_weighted_mean = (dy * pixel_area).sum(dim=('lon', 'lat')) / total_ocean_area
print(sst_weighted_mean)
dy = da2a[:,:,0:48].max(dim=['orbit'])
pixel_area = dA.where(dy.notnull())
total_ocean_area = pixel_area.sum(dim=('lon', 'lat'))
sst_weighted_mean = (dy * pixel_area).sum(dim=('lon', 'lat')) / total_ocean_area
print(sst_weighted_mean)

In [ ]:
# now lets add in some real data to the orbits
#fname = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/L4/GLOB/UKMO/OSTIA/2019/012/20190112-UKMO-L4HRfnd-GLOB-v01-fv02-OSTIA.nc.bz2'
#ds_sst = xr.open_dataset(fname)
ds_sst = ds_ecco.interp(lat=np.arange(-90,90,0.1),lon=np.arange(-180,180,0.1))
da_sst=[]
for i in range(48):
#    tem = np.where(da2[:,:,i]==1,ds_sst.analysed_sst[0,:,:].data,np.nan)
    tem = np.where(da2[:,:,i]==1,ds_sst.Theta.data,np.nan)
    tem = np.expand_dims(tem,2)
    tem = xr.DataArray(tem,name='sst',
                  coords={'lat':grid_def_lats,'lon':grid_def_lons,'orbit':[i]},
                  dims=('lat','lon','orbit'))
    da_sst.append(tem)
da_sst2 = xr.concat(da_sst, dim='orbit')

In [ ]:
da_sst2[:,:,0].plot()

In [ ]:
fig = plt.figure(figsize=(10, 8))
coord1 = 121
coord2 = 122
da2a = da2.fillna(0)
dy = da2a[:,:,7:10].sum(dim=['orbit'])
sst = da_sst2[:,:,9:12].max(['orbit'])-273.15
dy = dy.where(dy<0,sst)
dy = dy.where(ds_topo2.etopo_depth<0)  #mask land regions
dy = dy.where(dy>0,np.nan)
ax = plt.subplot(coord1,projection=ccrs.Orthographic(150, 10))
ax.imshow(img_white, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
dy.plot(ax=ax, transform=ccrs.PlateCarree(),cmap='jet',vmin=0,vmax=30,add_colorbar=False)
plt.savefig(adir_figs+'sst_data_orbit.png', dpi=100)

In [ ]:
# ecco
!pip install git+https://github.com/rabernat/xmitgcm.git@fix-llcreader-klevels-bug

In [ ]:
from xmitgcm import llcreader
model = llcreader.PleiadesLLC2160Model()
ds = model.get_dataset(varnames=['Theta'], type='latlon')